# 数据探索

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
sns.set_style('whitegrid')
import matplotlib.pyplot as plt
plt.style.use('ggplot')
# plt.rcParams[u'font.sans-serif'] = ['simhei']
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

from scipy import stats


## 训练集(train.csv)探索

In [ ]:
train = pd.read_csv('../data/train.csv')
train.head()

In [ ]:
train.describe()

In [ ]:
train.apply(lambda x: sum(x.isnull()))

在训练集中没有缺失值，因为给后续工作带来了方便。

## 变量分析

### 相关系数

In [ ]:
fig = plt.figure(figsize=(10,10))
coff = train.corr()
_ = sns.heatmap(coff, annot=True)

从相关系数的热力图，可以看出：
1. Sales和Customers呈正相关，这是符合逻辑的，更大的客流量带来了更多的销售额，如果没有顾客，销售额肯定为0；
2. Open与Sales和Customers也呈正相关，原因和上面一样；
3. 促销活动也会带来更大的销售额（正相关）
4. DayOfWeek与其他特征都是负相关，说明周末的销售额要小于工作日的销售额

### Sales

In [ ]:
# 观察sales的分布
fig = plt.figure(figsize=(15,5))
_ = plt.subplot(121)
sns.distplot(train['Sales'], fit=stats.norm)
plt.title('Sales')
_ = plt.subplot(122)
res = stats.probplot((train['Sales']), plot=plt)

从上图中可以看出，在训练集中存在销售额为0的情况,并且存在大量的离群值。<br>
整体上看，销售额不是正态分布，存在明显的长尾。需要对Sales进行对数变换。

### Customers V.S. Sales
逻辑上，顾客的数量应该与销售额呈正比。

In [ ]:
g = sns.FacetGrid(train, size=7)
_ = g.map(plt.scatter, 'Customers', 'Sales', alpha=0.7)

如上图所示，顾客数量与销售额呈正比，但是需要注意，存在部分的离群值。<br>
同时，当顾客数量为0时，销售额也为0，这是符合逻辑的。**因此，对于测试集中的数据，如果Customers=0，即可以预测Sales=0**。同时，销售额为0的情况下，极有可能是因为Open=0。

In [ ]:
train[train['Open']==0].describe()

从上面的表格，可以说明当Open=0是，Sales和Customers都是0，同时还能看出，大多数情况下（≥75%），星期天没有营业。

### DayOfWeek V.S. Sales

In [ ]:
fig = plt.figure(figsize=(15,5))
_ = sns.boxplot(x='DayOfWeek', y='Sales', data=train)

销售额在星期天的时候普遍明显低于其他时间，同时，销售额在星期一的中位数最高。

### Promo V.S. Sales

In [ ]:
fig = plt.figure(figsize=(15,5))
_ = sns.boxplot(x='Promo', y='Sales', data=train)

促销也会带来明显的销售额提升。

### SchoolHoliday V.S. Sales

In [ ]:
fig = plt.figure(figsize=(15,5))
_ = sns.boxplot(x='SchoolHoliday', y='Sales', data=train)

学校是否放假，对销售额的影响不大。

## 商店信息(store.csv)

In [ ]:
store = pd.read_csv('../data/store.csv')
store.head()

In [ ]:
store.describe()

In [ ]:
store.describe(include='object')

除Promo2, StoreType和Assortment外，其他特征都有缺失值存在。

In [ ]:
store.apply(lambda x: sum(x.isnull()))

### CompetitionDistance

In [ ]:
store.loc[store['CompetitionDistance'].isnull(), :]

当CompetitionDistance缺失时，其他缺失特征也存在。只有三个样本缺失CompetitionDistance，因此可以认为，这三家商店附近没有竞争对手。在数据集中，距离竞争对手最远的距离是75860，所以可以将缺失值设为99999。

In [ ]:
store['CompetitionDistance'] = store['CompetitionDistance'].fillna(99999)

### CompetitionOpenSinceMonth和CompetitionOpenSinceYear 
竞争对手开业年份和开业月份的缺失数量相同，可以设想应该是同一商店发生缺失。

In [ ]:
# 提取CompetitionOpenSinceMonth缺失的样本
nan_set = store.loc[store['CompetitionOpenSinceMonth'].isnull(), :]
print('CompetitionOpenSinceMonth 缺失样本数量为:\n%d'%len(nan_set))

# 计算CompetitionOpenSinceMonth缺失样本数量
print('\n同时CompetitionOpenSinceYear 缺失样本数量为:')
print(store.loc[nan_set.index, 'CompetitionOpenSinceYear'].isnull().sum())

可以判断，对同一商店，同时缺失竞争对手开业年份和开业月份，其中有三个商店还缺失竞争对手的距离信息。<br>
因为缺失的数量较多，所以不能直接忽略，对于这两个特征，有两种处理方式：
1. 将同时缺失距离，月份，年份的样本视为附近没有竞争对手，将距离填充为99999，日期填充为未来某一天；
1. 将缺失值用平均数或者中位数填充；
2. 将这两个特征当做分类变量处理，把缺失值作为单独一类处理；
4. 竞争对手存在与否，对商店的销售量存在一定的影响，可以结合销售额进行填充。

### Promo2SinceWeek， Promo2SinceYear和PromoInterval

In [ ]:
# 提取CompetitionOpenSinceMonth缺失的样本
nan_set = store.loc[store['Promo2SinceWeek'].isnull(), :]
print('Promo2SinceWeek 缺失样本数量为:\n%d'%len(nan_set))

# 计算Promo2SinceYear缺失样本数量
print('\nPromo2SinceYear 缺失样本数量为:')
print(store.loc[nan_set.index, 'Promo2SinceYear'].isnull().sum())

# 计算PromoInterval缺失样本数量
print('\nPromoInterval 缺失样本数量为:')
print(store.loc[nan_set.index, 'PromoInterval'].isnull().sum())

对于促销信息，缺失的也是同一商店。结合第一部分的分析促销对销售额有影响，因此与竞争对手信息一样，可以用平均值或者中位数对缺失值进行填充，也可以结合销售信息进行填充。

## 合并train.csv和store.csv

In [ ]:
dataset = pd.merge(train, store, on='Store', how='left')
dataset.head()

In [ ]:
# 提取没有缺失值的样本
non_null = dataset.dropna(axis=0).copy()
non_null.head()

In [ ]:
store_sales = pd.DataFrame(non_null[non_null['Sales'] != 0].groupby('Store')['Sales', 'CompetitionDistance'].mean(), columns=['Sales', 'CompetitionDistance'])
g = sns.FacetGrid(store_sales, size=8)
_ = g.map(plt.scatter, 'CompetitionDistance', 'Sales')
_ = plt.xlabel('Distance')
_ = plt.ylabel('Average Sales')